
# Preflight Tests — Stimulus Generator & Participant Schedule

Run these cells **before** spending money on API calls. They verify:
1) The *participant schedule* (`trials_participants.csv`) is well-formed.
2) File paths point to real images and match their declared condition/label.
3) **Shared-noise** guarantees hold (weak/strong use *identical* noise in mixture trials).
4) **Signal math** checks out (`combined - noise == signal`).
5) Basic distribution sanity (present rate ~ 0.5; strong assignment ~ uniform across P1–P3).

> Assumes you've already run `export_trials_with_participants()` in your `ImageGen.py`.


In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np

# Paths
TRIALS_PARTICIPANTS = Path("trials_participants.csv")
DATASET_ROOT = Path("perceptual_dataset_calibrated")

assert TRIALS_PARTICIPANTS.exists(), "Missing trials_participants.csv — run export_trials_with_participants() first."
assert DATASET_ROOT.exists(), f"Missing dataset root: {DATASET_ROOT}"
print("[ok] Paths look good.")


In [ ]:

# Load schedule and run basic structure checks
df = pd.read_csv(TRIALS_PARTICIPANTS)
need = {"TrialID","Block","ParticipantID","Truth","AssignedCondition","Image"}
assert need.issubset(df.columns), f"Missing columns. Need {need}, got {set(df.columns)}"

n_trials = df["TrialID"].nunique()
triplets_ok = (df.groupby("TrialID").size() == 3).all()
print(f"[data] rows={len(df)} unique TrialID={n_trials} triplets_ok={triplets_ok}")
assert triplets_ok, "Each TrialID must have exactly 3 participant rows"

display(df.head(6))


In [ ]:

# 1) Equal block sanity
eq = df[df["Block"]=="equal"].copy()
assert not eq.empty, "No equal block rows found"
assert (eq["AssignedCondition"]=="equal").all(), "Equal block rows must have AssignedCondition == 'equal'"

# Presence prior near 0.5 on equal block (use tolerance for small N)
p_eq = eq.drop_duplicates("TrialID")["Truth"].mean()
print(f"[equal] present rate ≈ {p_eq:.3f} (expect ~0.5)")
assert 0.3 <= p_eq <= 0.7, "Equal-block present rate looks off"


In [ ]:

# 2) Mixture absent: everyone sees same absent image (we schedule 'weak' path rows in CSV)
mix_abs = df[(df["Block"]=="mixture") & (df["Truth"]==0)].copy()
if mix_abs.empty:
    print("[mixture/absent] no absent rows found — skipping check")
else:
    assert (mix_abs["AssignedCondition"]=="weak").all(),         "Mixture-absent participant rows should consistently use 'weak' path in CSV (image is absent anyway)"
    # All 3 participants on the same TrialID should point to the same image path
    same_img = (mix_abs.groupby("TrialID")["Image"].nunique() == 1).all()
    print(f"[mixture/absent] all 3 participants share the same absent image: {same_img}")
    assert same_img, "Mixture-absent rows must reference the same image path per TrialID"


In [ ]:

# 3) Mixture present: exactly one strong and two weak per TrialID
mix_pre = df[(df["Block"]=="mixture") & (df["Truth"]==1)].copy()
if mix_pre.empty:
    print("[mixture/present] no present rows found — skipping check")
else:
    ct = (mix_pre.groupby(["TrialID","AssignedCondition"]).size()
          .unstack(fill_value=0))
    # Verify 1 strong + 2 weak per TrialID
    ok_mask = (ct.get("strong",0) == 1) & (ct.get("weak",0) == 2)
    ok_ratio = ok_mask.mean()
    print(f"[mixture/present] fraction of trials with 1 strong + 2 weak = {ok_ratio:.3f}")
    assert ok_mask.all(), "Each mixture-present TrialID must have exactly 1 strong and 2 weak rows"

    # Uniformity of strong assignment across participants (rough check)
    strong_pid_counts = (mix_pre[mix_pre["AssignedCondition"]=="strong"]
                         ["ParticipantID"].value_counts().sort_index())
    print("[mixture/present] strong assignment counts by ParticipantID:")
    display(strong_pid_counts)


In [ ]:

# 4) Path integrity: files exist and paths match cond/label expectations
from pathlib import Path

def path_looks_right(row):
    p = Path(row["Image"])
    if not p.exists():
        return False
    cond = row["AssignedCondition"]
    label = "present" if int(row["Truth"])==1 else "absent"
    # expected pattern: .../<cond>/<label>/combined/<cond>_<label>_<id>_combined.png
    return (f"/{cond}/{label}/combined/" in str(p).replace("\\","/") 
            and f"{cond}_{label}_" in p.name)

ok = df.apply(path_looks_right, axis=1)
missing = (~ok).sum()
print(f"[paths] total={len(ok)} bad={missing}")
assert ok.all(), "Some Image paths do not exist or do not match (cond/label)"


In [ ]:

# 5) Shared-noise tests: mixture-present and mixture-absent
import numpy as np

# helper to load noise npy given cond/label/tid
def noise_npy_path(cond, label, tid):
    return DATASET_ROOT/cond/label/"noise"/f"{cond}_{label}_{tid:03d}_noise.npy"

# Sample a few mixture-present TrialIDs
mix_pre_ids = mix_pre["TrialID"].drop_duplicates().tolist() if not mix_pre.empty else []
if mix_pre_ids:
    samp = mix_pre_ids[: min(5, len(mix_pre_ids))]
    for tid in samp:
        w_path = noise_npy_path("weak","present",tid)
        s_path = noise_npy_path("strong","present",tid)
        assert w_path.exists() and s_path.exists(), f"Missing noise npy for TrialID {tid}"
        w_noise = np.load(w_path); s_noise = np.load(s_path)
        assert np.allclose(w_noise, s_noise), f"Noise mismatch (weak vs strong) on TrialID {tid}"
    print("[noise] mixture-present: weak and strong share identical noise ✅")
else:
    print("[noise] no mixture-present trials to test")

# Sample a few mixture-absent TrialIDs
mix_abs_ids = mix_abs["TrialID"].drop_duplicates().tolist() if not mix_abs.empty else []
if mix_abs_ids:
    samp = mix_abs_ids[: min(5, len(mix_abs_ids))]
    for tid in samp:
        w_path = noise_npy_path("weak","absent",tid)
        s_path = noise_npy_path("strong","absent",tid)
        if w_path.exists() and s_path.exists():
            w_noise = np.load(w_path); s_noise = np.load(s_path)
            assert np.allclose(w_noise, s_noise), f"Absent noise mismatch on TrialID {tid}"
    print("[noise] mixture-absent: weak and strong share identical noise ✅")
else:
    print("[noise] no mixture-absent trials to test")


In [ ]:

# 6) Signal math: check that combined - noise == signal (within float tolerance)
def triple_paths(cond, label, tid):
    base = DATASET_ROOT/cond/label
    comb = base/"combined"/f"{cond}_{label}_{tid:03d}_combined.npy"
    noi  = base/"noise"/f"{cond}_{label}_{tid:03d}_noise.npy"
    sig  = base/"signal"/f"{cond}_{label}_{tid:03d}_signal.npy"
    return comb, noi, sig

def check_signal(cond, tid):
    comb, noi, sig = triple_paths(cond, "present", tid)
    assert comb.exists() and noi.exists() and sig.exists(), f"Missing npy for {cond} {tid}"
    c = np.load(comb); n = np.load(noi); s = np.load(sig)
    err = np.abs((c - n) - s).max()
    return err

checked_any = False
for cond in ["weak","strong","equal"]:
    # find present TrialIDs for this cond
    tids = df[(df["AssignedCondition"]==cond) & (df["Truth"]==1)]["TrialID"].unique().tolist()
    for tid in tids[:3]:
        err = check_signal(cond, tid)
        assert err < 1e-6, f"Signal mismatch for {cond} {tid} (max abs err={err})"
        checked_any = True
if checked_any:
    print("[signal] combined - noise == signal on sampled present images ✅")
else:
    print("[signal] no present images sampled for signal check")


In [ ]:

# 7) Distribution checks
eq_trials = df[df["Block"]=="equal"].drop_duplicates("TrialID")
mix_trials = df[df["Block"]=="mixture"].drop_duplicates("TrialID")
p_eq = eq_trials["Truth"].mean() if not eq_trials.empty else float('nan')
p_mix = mix_trials["Truth"].mean() if not mix_trials.empty else float('nan')
print(f"[rates] equal present rate:   {p_eq:.3f}")
print(f"[rates] mixture present rate: {p_mix:.3f}")

# Strong assignment uniformity across participants (rough)
if not mix_pre.empty:
    strong_pid = (mix_pre[mix_pre["AssignedCondition"]=="strong"]["ParticipantID"]
                  .value_counts().sort_index())
    print("[uniformity] strong counts by ParticipantID:")
    display(strong_pid)



## Next
If all assertions passed:
- Open **LLMRunner2.ipynb**.
- Set `DRY_RUN = True` (or `DRY_RUN=1` in env).
- Run a small subset first: `p1 = run_phase1(dfp, limit_trials=5)` then `p2 = run_phase2(p1)`.
- Inspect `API_files/phase1_results.csv` and `API_files/phase2_results.csv`.
